In [ ]:
"""
1. Create the VPC

gcloud compute networks create example-k8s --subnet-mode custom

"""

In [ ]:
"""
2. Create the k8s-nodes subnet in the example-k8s VPC network:

gcloud compute networks subnets create k8s-nodes \
--network example-k8s \
--range 10.240.0.0/24

"""

In [ ]:


"""
3 Create a firewall rule that allows internal communication across TCP, UDP, ICMP and IP in IP (used for the Calico overlay):

gcloud compute firewall-rules create example-k8s-allow-internal \
--allow tcp,udp,icmp,ipip \
--network example-k8s \
--source-ranges 10.240.0.0/24

"""

In [ ]:

"""
4.Create a firewall rule that allows external SSH, ICMP, and HTTPS:

gcloud compute firewall-rules create example-k8s-allow-external \
--allow tcp:22,tcp:6443,icmp \
--network example-k8s \
--source-ranges 0.0.0.0/0


"""

In [ ]:
"""
5. Create the controller VM


gcloud compute instances create controller1 \
  --async \
  --boot-disk-size 200GB \
  --can-ip-forward \
  --image-family ubuntu-2004-lts \
  --image-project ubuntu-os-cloud \
  --machine-type n1-standard-2 \
  --private-network-ip 10.240.0.12 \
  --scopes compute-rw,storage-ro,service-management,service-control,logging-write,monitoring \
  --subnet k8s-nodes \
  --zone asia-east1-a \
  --tags example-k8s,controller 

  """

In [ ]:
"""
Create three worker VMs.

for i in 0; do
  gcloud compute instances create worker-${i} \
  --async \
  --boot-disk-size 200GB \
  --can-ip-forward \
  --image-family ubuntu-2004-lts \
  --image-project ubuntu-os-cloud \
  --machine-type n1-standard-2 \
  --private-network-ip 10.240.0.2${i} \
  --scopes compute-rw,storage-ro,service-management,service-control,logging-write,monitoring \
  --subnet k8s-nodes \
  --zone asia-east1-a \
  --tags example-k8s,worker
done

"""

In [ ]:


"""


7 Install Docker on the controller VM and each worker VM. On each VM run:

sudo apt update
sudo apt install -y docker.io
sudo systemctl enable docker.service
sudo apt install -y apt-transport-https curl
sudo swapoff -a && sudo sed -i '/ swap / s/^\(.*\)$/#\1/g' /etc/fstab
"""

In [ ]:
"""
1 Install kubeadm, kubelet, and kubectl on each worker node and the controller node (see kubeadm docs for more details).

curl -fsSL https://dl.k8s.io/apt/doc/apt-key.gpg | sudo apt-key add -
cat <<EOF | sudo tee /etc/apt/sources.list.d/kubernetes.list
deb https://apt.kubernetes.io/ kubernetes-xenial main
EOF
sudo apt-get update
sudo apt-get install -y kubelet kubeadm kubectl
sudo apt-mark hold kubelet kubeadm kubectl

"""

In [ ]:
"""
2 Create the controller node of a new cluster.

sudo kubeadm init --pod-network-cidr 192.168.0.0/16

--skip-phases=addon/kube-proxy 
"""
# Should output somehting. This ahs to be used in step 4

# sudo kubeadm join 10.240.0.11:6443 --token <token> --discovery-token-ca-cert-hash sha256:<hash>

In [ ]:
"""
3 Set up kubectl for the Ubuntu user.

Connect to the controller VM, and run the following commands:

mkdir -p $HOME/.kube
sudo cp -i /etc/kubernetes/admin.conf $HOME/.kube/config
sudo chown $(id -u):$(id -g) $HOME/.kube/config

"""

In [ ]:
"""
4. Join your worker nodes to the controller node.
Use the output of step 2

sudo kubeadm join 10.240.0.11:6443 --token 40on7w.idqo4gzq8asfj6qa \
        --discovery-token-ca-cert-hash sha256:d48ee03910784df4c577ed69f22e1b2c1dd0fc3715010b0ced791a01083e763e

"""

In [ ]:
# kubectl get nodes
# kubectl get all -A
# kubectl describe nodes
# on controller vm to get the status

sudo systemctl restart kubelet
kubectl get services -n kubeflow

kubectl port-forward -n kubeflow svc/ml-pipeline-ui 8080:80


![Alt text](image.png)

In [ ]:
curl https://docs.projectcalico.org/manifests/calico.yaml -O
kubectl apply -f calico.yaml

kubectl apply -f https://raw.githubusercontent.com/coreos/flannel/master/Documentation/kube-flannel.yml

In [ ]:
"""
export PIPELINE_VERSION=2.0.5
kubectl apply -k "github.com/kubeflow/pipelines/manifests/kustomize/cluster-scoped-resources?ref=$PIPELINE_VERSION"
kubectl wait --for condition=established --timeout=60s crd/applications.app.k8s.io
kubectl apply -k "github.com/kubeflow/pipelines/manifests/kustomize/env/dev?ref=$PIPELINE_VERSION"
"""

In [ ]:

sudo adduser containiq
sudo usermod -aG sudo containiq
su - containiq
sudo apt update && sudo apt upgrade -y
sudo swapoff -a && sudo sed -i '/ swap / s/^\(.*\)$/#\1/g' /etc/fstab
sudo hostnamectl set-hostname primary

In [ ]:
sudo adduser containiq
sudo usermod -aG sudo containiq
su - containiq
sudo apt update && sudo apt upgrade -y
sudo swapoff -a && sudo sed -i '/ swap / s/^\(.*\)$/#\1/g' /etc/fstab
sudo hostnamectl set-hostname worker

In [ ]:
sudo nano /etc/hosts
sudo reboot


In [ ]:
sudo apt install ca-certificates curl gnupg lsb-release
curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo gpg --dearmor -o /usr/share/keyrings/docker-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/usr/share/keyrings/docker-archive-keyring.gpg] https://download.docker.com/linux/ubuntu $(lsb_release -cs) stable" | sudo tee /etc/apt/sources.list.d/docker.list > /dev/null
sudo apt update && sudo apt install docker-ce docker-ce-cli containerd.io -y
sudo systemctl start docker && sudo systemctl enable docker 